In [1]:
%run stdPackages.ipynb
os.chdir(d['py'])
import US_EulerModel_main, US_EulerModel_c, US_EulerModel_policy
os.chdir(d['curr'])

# Social Security Design for US (Euler, finite horizon): 
## Identification of structural parameters $(\xi, \theta)$ for 2010 model

This notebook considers the model for the US where all household types are "Euler types" (e.g. no hand-to-mouth or unemployed). We search the parameter space of the coefficient of relative risk aversion ($\theta$) and frisch elasticity $(\xi)$ of households to identify combinations that ensures the model replicates the choice of pension characteristics of 2010.

The model uses a finite horizon version to solve for the CRRA model. We revisit the robustness of results in other notebooks.

## 1. Data and settings

Data on inputs:

In [2]:
t0date = 2010
%run USEuler_Data.ipynb

Initial parameter values and settings:

In [3]:
ω0 = 1.75
ngrid = 50

Grid specification for searches over $\rho, \xi$:

In [4]:
ξ0, ξLower, nξ = .5, .1, 9
ρ0, ρLower, nρ = 2.5, 1, 11

Initialize model:

In [5]:
m = US_EulerModel_main.Model(ngrid = ngrid, ξ = ξ0, ρ = ρ0, ω = ω0, **data)

Specify others settings: Size of grids to use in grid search methods and the style of solution to use (see ```USEuler_Policy.ipynb``` on the different methods):

In [6]:
m.ESC.grids['τ'] = m.ESC.nGrids('τ', 101) # create standard grid with 101 steps for grid searches for τ
m.ESC.grids['κ'] = m.ESC.nGrids('κ', 51) # create standard grid with 51 steps for grid searches for κ
m.ESC.kwargs_t = {'style': 'VeryRobust'} # use the "VeryRobust" style of solving the model.
m.ESC.kwargs_T = {'style': 'VeryRobust'} # use the "VeryRobust" style of solving the model in terminal state.

## 2. Calibration

We target a range of characteristics including distributions in working hours and wages, the 30 years interest rate $(R_t)$, and the endogenous choice of pension + social security taxes $\tau_t$. On top of this, we want to replicate the endogenous choice of system characteristics ($\kappa_t$) that determines the share of pensions that are "universal". The distribution of wages and working hours can be in a very simple way by adjusting $\eta_i, X_i$. This calibration can be done outside the numerical simulation of the model *given* the labor elasticity $\xi$. 

The second set of targets $\tau_t, R_t$ can be identified by adjusting the political weight of retirees $\omega$ and the discount factor $\beta$; This identification, however, is only done by simulating model solutions until the targets are reached. Thus, this is a somewhat costly calibration. 

Finally, the choice of $\kappa_t$ is numerically very challenging to identify. We essentially have two parameters that can help us identify the choice of the design of the system: The CRRA preference $\rho$ and the labor elasticity $\xi$. As we can see in the following, there is a small subset of the parameter space for $(\xi, \rho)$ that permits an interior solution. To get at this, we start by solving the model on a grid of both $(\rho, \xi)$: For each gridpoint, we define the "objective" as the difference between the target policy design $\overline{\kappa}$ and the endogenous choice $\kappa_t$. What we need from the initial gridsearch is at least one level of $\rho_n$ (or $\xi$) where the objective on the grid of $\xi$ (or $\rho$) the objective value changes sign: A sign change allows us to apply a golden-section like search for a solution value of $\xi$ (given $\rho$). We can then proceed in a number of ways, depending on how much of the solution space (for $\xi, \rho$) we are interested in:
* We can simply use the part of the solution space on the initial grid of $(\rho, \xi)$, where we observe a sign change.
* We can start with the solution space on the initial grid, where we observe a sign change. Next, we can take the first and last value of $\rho$, where we found a sign change on the grid of $\xi$. Then, we can form a new grid between points where we *did* find a sign change and the nearest gridpoint where we did not. 

*Grids:*

In [7]:
β0 = m.US_βinv()
ρgrid = np.round(np.linspace(ρ0, ρLower, nρ), 2)
ξgrid = np.round(np.linspace(ξ0, ξLower, nξ), 3)

*Start from pickle instead:*

In [8]:
# with open(os.path.join(d['data'], 'USEuler2010_gridsearch'), "rb") as file:
#     solsGrid = pickle.load(file)

To be able to "start" the model from a reasonable parameter range, let us start by solving on a grid of $\xi$:

In [9]:
sols0 = m.USCal_OnGrid(ξgrid, data['t0'], var = 'ξ')

Then, given this, solve on a grid of $\xi$ for each step. Create a "break" variable as follows: If there has been no solution with a sign change on the grid of $\xi$ yet, keep iterating. The first time a sign change is recorded update local to ```solved```. Then, whenever there is no longer a sign change, stop iteration:

In [10]:
solved = False
solsGrid = dict.fromkeys(ξgrid)
for i in range(len(ξgrid)):
    m.db['ξ'] = ξgrid[i]
    m.US_Xi()
    m.db['ω'] = sols0['ω'][i]
    m.db['βi'] = m.US_β(sols0['β'][i])
    solsGrid[m.db['ξ']] = {'ρ' : ρgrid, 'obj': np.empty(ρgrid.size)}
    solsGrid[m.db['ξ']].update({k: solsGrid[m.db['ξ']]['obj'].copy() for k in ('ω','β')})
    for j in range(ρgrid.size):
        solsGrid[m.db['ξ']] = m.USCal_OnGrid_i(data['t0'], solsGrid[m.db['ξ']], j, var = 'ρ')
    # solsGrid[ξgrid[i]] = m.USCal_OnGrid(ρgrid, data['t0'], var = 'ρ')
    if solved is False:
        if np.nonzero(np.diff(np.sign(solsGrid[ξgrid[i]]['obj'])) != 0)[0].size>0:
            solved = True
    else:
        if np.nonzero(np.diff(np.sign(solsGrid[ξgrid[i]]['obj'])) != 0)[0].size == 0:
            break

Store data:

In [11]:
with open(os.path.join(d['data'], 'USEuler2010_gridsearch'), "wb") as file:
    pickle.dump(solsGrid, file)

### 2.1. Search for solutions on relevant part of grid.

Remove ```None``` entries and only keep sign changes:

In [12]:
solsSignChange = {k: v for k,v in solsGrid.items() if v}
solsSignChange = {k: v for k,v in solsSignChange.items() if np.nonzero(np.diff(np.sign(v['obj'])) != 0)[0].size>0}

Solve on this grid:

In [13]:
%%time
cals = dict.fromkeys(solsSignChange)
for k,v in solsSignChange.items():
    m.db['ξ'] = k
    m.US_Xi() # update Xi based on ξ
    sc = m.USCal_SCidx(v) # idx for sign change
    m.db['ω'] = v['ω'][sc]
    m.db['βi'] = m.US_β(v['β'][sc])
    cals[k] = m.USCal_GoldenSection(v['ρ'][sc:sc+2], data['t0'], n = 2, tol = 1e-5, iterMax = 5, var = 'ρ')
    print(k)

0.5
0.45
0.4
0.35
0.3
0.25
0.2
0.15
0.1
CPU times: total: 23min 47s
Wall time: 20min 25s


Calibration space:

In [14]:
pd.Series([cals[k]['ρ'] for k in cals], index = pd.Index(cals, name = 'ξ'))

ξ
0.50    2.071868
0.45    2.075518
0.40    2.079227
0.35    2.082872
0.30    2.086547
0.25    2.090237
0.20    2.093909
0.15    2.097570
0.10    2.101245
dtype: float64

### 3. Export:

In [15]:
with open(os.path.join(d['data'], 'USEuler2010_cals'), "wb") as file:
    pickle.dump(cals, file)
with open(os.path.join(d['data'], 'mUSEuler2010'), "wb") as file:
    pickle.dump(m, file)